In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("exercise").getOrCreate()
spark

In [39]:
from google.colab import drive
drive.mount('/content/drive')
df = spark.read.csv("/content/drive/MyDrive/Sales_Dataset__500_Records_.csv", header=True, inferSchema=True)
df.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+-------+------------------+---------------+------+----------+--------------+--------+-----------------+-----------+-------------+
|OrderID|      CustomerName|ProductCategory|Amount| OrderDate|DeliveryStatus|Discount|             City|PaymentMode|CustomerSince|
+-------+------------------+---------------+------+----------+--------------+--------+-----------------+-----------+-------------+
|   2824|     Donald Walker|          Books|783.04|2024-12-26|      Returned|    0.15|     Lake Joyside|Credit Card|   2020-10-15|
|   7912|      Brandon Hall|      Groceries| 905.0|2024-09-12|     Cancelled|    0.03|    New Jamesside|     Wallet|   2022-03-15|
|   4611|      Donald Booth|        Fashion|657.96|2025-01-12|      Returned|    0.01|     Lake Roberto|     Wallet|   2021-08-07|
|   3547|    Phillip Garcia|        Fashion|606.89|2024-03-24|      Returned|    0.15

1. DataFrame Creation and Inspection

In [10]:
df.tail(5)

[Row(OrderID=2930, CustomerName='Jaime Harris', ProductCategory='Fashion', Amount=680.0, OrderDate=datetime.date(2025, 2, 16), DeliveryStatus='Returned', Discount=0.2, City='Robertville', PaymentMode='Cash', CustomerSince=datetime.date(2021, 1, 28)),
 Row(OrderID=7980, CustomerName='Dawn Wyatt', ProductCategory='Fashion', Amount=285.32, OrderDate=datetime.date(2024, 4, 26), DeliveryStatus='Cancelled', Discount=0.06, City='Cherylfurt', PaymentMode='UPI', CustomerSince=datetime.date(2021, 4, 14)),
 Row(OrderID=7770, CustomerName='Kristin White', ProductCategory='Groceries', Amount=792.11, OrderDate=datetime.date(2024, 8, 10), DeliveryStatus='Returned', Discount=0.07, City='Kingport', PaymentMode='UPI', CustomerSince=datetime.date(2022, 11, 16)),
 Row(OrderID=6641, CustomerName='Jennifer Taylor', ProductCategory='Toys', Amount=578.49, OrderDate=datetime.date(2025, 4, 3), DeliveryStatus='Delivered', Discount=0.1, City='Lake Jerryburgh', PaymentMode='Cash', CustomerSince=datetime.date(2020,

In [22]:
df.show(-5)

+-------+------------+---------------+-----------+---------+--------------+--------+----+-----------+-----------+
|OrderID|CustomerName|ProductCategory|OrderAmount|OrderDate|DeliveryStatus|Discount|City|PaymentMode|FinalAmount|
+-------+------------+---------------+-----------+---------+--------------+--------+----+-----------+-----------+
+-------+------------+---------------+-----------+---------+--------------+--------+----+-----------+-----------+
only showing top 0 rows



In [9]:
df.printSchema()
df.dtypes

root
 |-- OrderID: integer (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- ProductCategory: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- DeliveryStatus: string (nullable = true)
 |-- Discount: double (nullable = true)
 |-- City: string (nullable = true)
 |-- PaymentMode: string (nullable = true)
 |-- CustomerSince: date (nullable = true)



[('OrderID', 'int'),
 ('CustomerName', 'string'),
 ('ProductCategory', 'string'),
 ('Amount', 'double'),
 ('OrderDate', 'date'),
 ('DeliveryStatus', 'string'),
 ('Discount', 'double'),
 ('City', 'string'),
 ('PaymentMode', 'string'),
 ('CustomerSince', 'date')]

2.Selection, Renaming, and Filtering

In [11]:
df.select("OrderID", "CustomerName", "Amount").show()


+-------+------------------+------+
|OrderID|      CustomerName|Amount|
+-------+------------------+------+
|   2824|     Donald Walker|783.04|
|   7912|      Brandon Hall| 905.0|
|   4611|      Donald Booth|657.96|
|   3547|    Phillip Garcia|606.89|
|   8527|      Valerie Gray| 77.87|
|   4150|       Amber Perez|352.37|
|   5554|        Roy Martin|148.33|
|   2169|    Carolyn Daniel| 14.09|
|   6313|       Patty Perez| 79.83|
|   6155|Jonathan Wilkerson|882.68|
|   9830|       Kevin Hurst|870.55|
|   9085| Anthony Rodriguez|921.73|
|   2040|     Kyle Mcdonald|327.52|
|   6573|    Jeffrey Chavez|676.02|
|   2743|  Elizabeth Fowler| 47.06|
|   9837|     Tammy Sellers| 46.15|
|   6038|     David Bradley|348.51|
|   3060|       John Pierce|362.09|
|   4295|   Jennifer Powers|684.26|
|   5061|    George Chapman|251.89|
+-------+------------------+------+
only showing top 20 rows



In [12]:
df = df.withColumnRenamed("Amount", "OrderAmount")


In [14]:
from pyspark.sql.functions import col
df.filter(col("OrderAmount") > 500).show()

+-------+------------------+---------------+-----------+----------+--------------+--------+-----------------+-----------+-------------+
|OrderID|      CustomerName|ProductCategory|OrderAmount| OrderDate|DeliveryStatus|Discount|             City|PaymentMode|CustomerSince|
+-------+------------------+---------------+-----------+----------+--------------+--------+-----------------+-----------+-------------+
|   2824|     Donald Walker|          Books|     783.04|2024-12-26|      Returned|    0.15|     Lake Joyside|Credit Card|   2020-10-15|
|   7912|      Brandon Hall|      Groceries|      905.0|2024-09-12|     Cancelled|    0.03|    New Jamesside|     Wallet|   2022-03-15|
|   4611|      Donald Booth|        Fashion|     657.96|2025-01-12|      Returned|    0.01|     Lake Roberto|     Wallet|   2021-08-07|
|   3547|    Phillip Garcia|        Fashion|     606.89|2024-03-24|      Returned|    0.15| West Melanieview|     Wallet|   2020-08-08|
|   6155|Jonathan Wilkerson|        Fashion|    

In [17]:
df.filter(col("City") == "New York").show()

+-------+------------+---------------+-----------+---------+--------------+--------+----+-----------+-------------+
|OrderID|CustomerName|ProductCategory|OrderAmount|OrderDate|DeliveryStatus|Discount|City|PaymentMode|CustomerSince|
+-------+------------+---------------+-----------+---------+--------------+--------+----+-----------+-------------+
+-------+------------+---------------+-----------+---------+--------------+--------+----+-----------+-------------+



3. Data Manipulation

In [18]:
df = df.drop("CustomerSince")

In [19]:
df = df.withColumn("FinalAmount", col("OrderAmount") - (col("OrderAmount") * col("Discount")))

In [20]:
df.orderBy(col("FinalAmount").desc()).show()

+-------+-----------------+---------------+-----------+----------+--------------+--------+-----------------+-----------+-----------------+
|OrderID|     CustomerName|ProductCategory|OrderAmount| OrderDate|DeliveryStatus|Discount|             City|PaymentMode|      FinalAmount|
+-------+-----------------+---------------+-----------+----------+--------------+--------+-----------------+-----------+-----------------+
|   5573|   Jordan Frazier|          Books|     981.05|2025-03-19|     Cancelled|    0.02|      Sheilaville|       Cash|          961.429|
|   8474|      Heidi Brown|    Electronics|     968.91|2023-11-23|     Cancelled|    0.02|       Riverafort|       Cash|         949.5318|
|   8889|      Karen Garza|          Books|      998.3|2024-10-17|     Cancelled|    0.06|       Johnsonton|Credit Card|938.4019999999999|
|   2127|     Jaclyn Moore|      Groceries|     933.32|2025-03-11|      Returned|    0.01|      Cherylhaven|       Cash|         923.9868|
|   9806|    Samantha Gill|

In [26]:
from pyspark.sql.functions import expr
df = df.replace("Cancelled", "Order Cancelled", subset=["DeliveryStatus"])
df.select("DeliveryStatus").distinct().show()

+---------------+
| DeliveryStatus|
+---------------+
|       Returned|
|      Delivered|
|Order Cancelled|
|        Pending|
+---------------+



4. Aggregations and GroupBy

In [27]:
df.groupBy("DeliveryStatus").count().show()

+---------------+-----+
| DeliveryStatus|count|
+---------------+-----+
|       Returned|  117|
|      Delivered|  119|
|Order Cancelled|  149|
|        Pending|  115|
+---------------+-----+



In [28]:
from pyspark.sql.functions import avg
df.groupBy("ProductCategory").agg(avg("OrderAmount").alias("AvgAmount")).show()


+---------------+------------------+
|ProductCategory|         AvgAmount|
+---------------+------------------+
|        Fashion| 500.6308235294116|
|      Groceries|459.51786407766957|
|    Electronics|           551.745|
|          Books| 568.6003773584907|
|           Toys| 534.2837499999999|
+---------------+------------------+



In [29]:
from pyspark.sql.functions import sum
df.groupBy("City").agg(sum("FinalAmount").alias("TotalSales")).show()

+----------------+------------------+
|            City|        TotalSales|
+----------------+------------------+
|     Ramseymouth| 738.2281999999999|
|East Edwardshire|230.09539999999998|
|      Thomasberg| 644.3563999999999|
|     Laurenville|          321.9384|
| South Colinstad|           629.016|
|    Lake Douglas|           780.072|
|   Williamsmouth|            9.8098|
|      Gordonport|          381.0926|
|  West Dawnmouth|            10.496|
|        Seanbury|          627.0803|
|     Sheilaville|           961.429|
|       Mollybury|            222.02|
|       Lisaville|43.405499999999996|
| Lake Jerrymouth|          395.9298|
|       Perezfort|            734.04|
|Port Nicoleshire|111.03739999999999|
|  South Samantha|          169.8004|
|     Port Willie|          780.2487|
|     Waltersfort|           386.967|
|       Youngbury|           268.524|
+----------------+------------------+
only showing top 20 rows



 5. Null Handling & Conditional Tagging

In [31]:
from pyspark.sql.functions import when
from pyspark.sql.functions import lit
df_with_nulls = df.withColumn("City", when(col("City") == "New York", lit(None)).otherwise(col("City")))


In [32]:
df_with_nulls.fillna({"City": "Unknown"}).show()
df_with_nulls.dropna(subset=["City"]).show()

+-------+------------------+---------------+-----------+----------+---------------+--------+-----------------+-----------+------------------+
|OrderID|      CustomerName|ProductCategory|OrderAmount| OrderDate| DeliveryStatus|Discount|             City|PaymentMode|       FinalAmount|
+-------+------------------+---------------+-----------+----------+---------------+--------+-----------------+-----------+------------------+
|   2824|     Donald Walker|          Books|     783.04|2024-12-26|       Returned|    0.15|     Lake Joyside|Credit Card|           665.584|
|   7912|      Brandon Hall|      Groceries|      905.0|2024-09-12|Order Cancelled|    0.03|    New Jamesside|     Wallet|            877.85|
|   4611|      Donald Booth|        Fashion|     657.96|2025-01-12|       Returned|    0.01|     Lake Roberto|     Wallet|          651.3804|
|   3547|    Phillip Garcia|        Fashion|     606.89|2024-03-24|       Returned|    0.15| West Melanieview|     Wallet|          515.8565|
|   85

In [33]:
df = df.withColumn("HighValue", when(col("OrderAmount") > 800, "Yes").otherwise("No"))
df.select("CustomerName", "OrderAmount", "HighValue").show()

+------------------+-----------+---------+
|      CustomerName|OrderAmount|HighValue|
+------------------+-----------+---------+
|     Donald Walker|     783.04|       No|
|      Brandon Hall|      905.0|      Yes|
|      Donald Booth|     657.96|       No|
|    Phillip Garcia|     606.89|       No|
|      Valerie Gray|      77.87|       No|
|       Amber Perez|     352.37|       No|
|        Roy Martin|     148.33|       No|
|    Carolyn Daniel|      14.09|       No|
|       Patty Perez|      79.83|       No|
|Jonathan Wilkerson|     882.68|      Yes|
|       Kevin Hurst|     870.55|      Yes|
| Anthony Rodriguez|     921.73|      Yes|
|     Kyle Mcdonald|     327.52|       No|
|    Jeffrey Chavez|     676.02|       No|
|  Elizabeth Fowler|      47.06|       No|
|     Tammy Sellers|      46.15|       No|
|     David Bradley|     348.51|       No|
|       John Pierce|     362.09|       No|
|   Jennifer Powers|     684.26|       No|
|    George Chapman|     251.89|       No|
+----------

6. Date & Time Functions

In [34]:
from pyspark.sql.functions import year, month, current_date, datediff
df.select("OrderDate", year("OrderDate").alias("Year"), month("OrderDate").alias("Month")).show()


+----------+----+-----+
| OrderDate|Year|Month|
+----------+----+-----+
|2024-12-26|2024|   12|
|2024-09-12|2024|    9|
|2025-01-12|2025|    1|
|2024-03-24|2024|    3|
|2024-08-04|2024|    8|
|2024-01-13|2024|    1|
|2024-03-04|2024|    3|
|2023-10-07|2023|   10|
|2023-06-27|2023|    6|
|2024-10-14|2024|   10|
|2024-04-08|2024|    4|
|2024-10-02|2024|   10|
|2023-12-15|2023|   12|
|2024-12-14|2024|   12|
|2024-09-18|2024|    9|
|2024-09-10|2024|    9|
|2023-08-03|2023|    8|
|2023-12-25|2023|   12|
|2024-03-19|2024|    3|
|2023-11-28|2023|   11|
+----------+----+-----+
only showing top 20 rows



In [41]:
from pyspark.sql.functions import to_date, datediff, current_date

df = df.withColumn("LoyaltyYears", datediff(current_date(), col("CustomerSince")) / 365)
df.select("CustomerName", "CustomerSince", "LoyaltyYears").show()

+------------------+-------------+------------------+
|      CustomerName|CustomerSince|      LoyaltyYears|
+------------------+-------------+------------------+
|     Donald Walker|   2020-10-15| 4.635616438356164|
|      Brandon Hall|   2022-03-15| 3.221917808219178|
|      Donald Booth|   2021-08-07| 3.824657534246575|
|    Phillip Garcia|   2020-08-08| 4.821917808219178|
|      Valerie Gray|   2022-11-15|2.5506849315068494|
|       Amber Perez|   2022-01-13| 3.389041095890411|
|        Roy Martin|   2023-04-29|2.0986301369863014|
|    Carolyn Daniel|   2021-05-09| 4.071232876712329|
|       Patty Perez|   2021-04-25| 4.109589041095891|
|Jonathan Wilkerson|   2021-06-20| 3.956164383561644|
|       Kevin Hurst|   2022-08-02|2.8383561643835615|
| Anthony Rodriguez|   2022-12-15|2.4684931506849317|
|     Kyle Mcdonald|   2021-07-21| 3.871232876712329|
|    Jeffrey Chavez|   2022-07-30|2.8465753424657536|
|  Elizabeth Fowler|   2021-02-07|  4.32054794520548|
|     Tammy Sellers|   2021-

7. Joins and Unions

In [42]:
region_data = [("New York", "East"), ("Los Angeles", "West"), ("Chicago", "Central")]
region_df = spark.createDataFrame(region_data, ["City", "Region"])


In [43]:
df.join(region_df, on="City", how="inner").show()
df.join(region_df, on="City", how="left").show()

+----+-------+------------+---------------+------+---------+--------------+--------+-----------+-------------+-----------------+------------+------+
|City|OrderID|CustomerName|ProductCategory|Amount|OrderDate|DeliveryStatus|Discount|PaymentMode|CustomerSince|CustomerSinceDate|LoyaltyYears|Region|
+----+-------+------------+---------------+------+---------+--------------+--------+-----------+-------------+-----------------+------------+------+
+----+-------+------------+---------------+------+---------+--------------+--------+-----------+-------------+-----------------+------------+------+

+-----------------+-------+------------------+---------------+------+----------+--------------+--------+-----------+-------------+-----------------+------------------+------+
|             City|OrderID|      CustomerName|ProductCategory|Amount| OrderDate|DeliveryStatus|Discount|PaymentMode|CustomerSince|CustomerSinceDate|      LoyaltyYears|Region|
+-----------------+-------+------------------+-------

In [44]:
df_2023 = df.filter(year("OrderDate") == 2023)
df_2024 = df.filter(year("OrderDate") == 2024)
df_union = df_2023.union(df_2024)
df_union.show()

+-------+----------------+---------------+------+----------+--------------+--------+-----------------+-----------+-------------+-----------------+------------------+
|OrderID|    CustomerName|ProductCategory|Amount| OrderDate|DeliveryStatus|Discount|             City|PaymentMode|CustomerSince|CustomerSinceDate|      LoyaltyYears|
+-------+----------------+---------------+------+----------+--------------+--------+-----------------+-----------+-------------+-----------------+------------------+
|   2169|  Carolyn Daniel|    Electronics| 14.09|2023-10-07|     Delivered|    0.25|         Grayside|Credit Card|   2021-05-09|       2021-05-09| 4.071232876712329|
|   6313|     Patty Perez|      Groceries| 79.83|2023-06-27|     Cancelled|    0.12|      Richardland|Credit Card|   2021-04-25|       2021-04-25| 4.109589041095891|
|   2040|   Kyle Mcdonald|           Toys|327.52|2023-12-15|      Returned|    0.06|Lake Jenniferside|     Wallet|   2021-07-21|       2021-07-21| 3.871232876712329|
|   

 8. Complex JSON Simulation

In [45]:
from pyspark.sql.functions import to_json, struct, get_json_object, explode
json_df = df.withColumn("order_json", to_json(struct([col(c) for c in df.columns])))
json_df.select("order_json").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_json                                                                                                                                                                                                                                                                                                        |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"OrderID":2824,"CustomerName":"Donald Walker","ProductCategory":"Books"

In [46]:
json_df.select(get_json_object("order_json", "$.CustomerName").alias("Customer")).show()


+------------------+
|          Customer|
+------------------+
|     Donald Walker|
|      Brandon Hall|
|      Donald Booth|
|    Phillip Garcia|
|      Valerie Gray|
|       Amber Perez|
|        Roy Martin|
|    Carolyn Daniel|
|       Patty Perez|
|Jonathan Wilkerson|
|       Kevin Hurst|
| Anthony Rodriguez|
|     Kyle Mcdonald|
|    Jeffrey Chavez|
|  Elizabeth Fowler|
|     Tammy Sellers|
|     David Bradley|
|       John Pierce|
|   Jennifer Powers|
|    George Chapman|
+------------------+
only showing top 20 rows



 9. Applying Functions (UDF)

In [48]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def tag_order(amount):
    if amount >= 800:
        return "Big"
    elif amount >= 300:
        return "Medium"
    else:
        return "Small"
tag_udf = udf(tag_order, StringType())
df = df.withColumn("OrderSize", tag_udf(col("Amount")))
df.select("Amount", "OrderSize").show()


+------+---------+
|Amount|OrderSize|
+------+---------+
|783.04|   Medium|
| 905.0|      Big|
|657.96|   Medium|
|606.89|   Medium|
| 77.87|    Small|
|352.37|   Medium|
|148.33|    Small|
| 14.09|    Small|
| 79.83|    Small|
|882.68|      Big|
|870.55|      Big|
|921.73|      Big|
|327.52|   Medium|
|676.02|   Medium|
| 47.06|    Small|
| 46.15|    Small|
|348.51|   Medium|
|362.09|   Medium|
|684.26|   Medium|
|251.89|    Small|
+------+---------+
only showing top 20 rows

